In [1]:
import os 
import json
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
import requests 
from newspaper import Article


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()


try:
    response = session.get(article_url, headers=headers, timeout=10)

    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()

        print(f"Title: {article.title}")
        print(f"Text: {article.text}")
    else:
        print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Ryan Daws is a senior editor at TechForge Media with over a decade of experience in crafting compelling narratives and making complex topics accessible. His articles and interviews with industry leaders have earned him recognition as a key influencer by organisations like Onalytica. Under his leadership, publications have been praised by analyst firms such as Forrester for their excellence and performance. Connect with him on X (@gadget_ry) or Mastodon (@gadgetry@techhub.social)

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and 

### Adding Few Shot Prompting

In [9]:
from langchain.schema import (
    HumanMessage
)


article_title = article.title
article_text = article.text


template = """

As an advanced AI, you've been tasked to summarize online articles into bulleted points. here are a few examples of how you've done this in the past:


Example 1:
Original Article: 'The Effects of climate Change'
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.



Example 2:
Original Article: 'The Evolution of Artificial Intelligence'
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

===========
Title: {article_title}

{article_text}
===========

Please provide a summarized version of the article in a bulleted list format
"""


prompt = template.format(article_title=article.title, article_text=article.text)
messages = [HumanMessage(content=prompt)]

In [11]:
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0)

summary = chat(messages)
print(summary.content)

- Meta (formerly Facebook) has announced the development of the AI Research SuperCluster (RSC), which it claims will be the world's fastest AI supercomputer upon completion.
- The RSC is designed to train large natural language processing (NLP) and computer vision models, with capabilities to handle trillions of parameters.
- Expected to be fully operational by mid-2022, RSC aims to facilitate advancements in real-time voice translations and augmented reality (AR) games among other applications.
- Once completed, RSC will be significantly faster than Meta's current computing infrastructure, being 20x faster than their V100-based clusters and improving training times for large-scale NLP workflows by 3x.
- The supercomputer incorporates enhanced security and privacy controls, allowing Meta to utilize real-world data from its production systems for training purposes.
- Meta believes RSC will play a crucial role in developing technologies for the metaverse, where AI-driven applications wil

### Adding parsers to output

In [12]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List



class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="""Bulleted list summary of the article""")
        
    # validating whether the generated summary has at least three lines
    @validator('summary', allow_reuse=True)
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("""Generated summary has less than three bullet points!""")
            
        return list_of_lines
    
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

In [14]:
from langchain.prompts import PromptTemplate

template = """
You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.


===============
Title: {article_title}
{article_text}

===============

{format_instructions}
"""



prompt_template = PromptTemplate(
template = template,
input_variables=["article_title", "article_text"],
partial_variables={"format_instructions": parser.get_format_instructions()}

)

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain


model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

chain = LLMChain(llm=model, prompt=prompt_template)

output = chain.run({"article_title":article_title, "article_text":article_text})

parsed_output = parser.parse(output)


Meta claims its new AI supercomputer will set records


In [19]:
print(parsed_output.summary)

["Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that it claims will be the world's fastest.", "RSC is still under construction but Meta's researchers have already started using it for training large natural language processing (NLP) and computer vision models.", 'Once fully built in mid-2022, RSC is expected to be the fastest supercomputer in the world and capable of training models with trillions of parameters.', 'Meta aims to use RSC to develop new AI systems for applications like real-time voice translations and AR games in the metaverse.', "The supercomputer is projected to be 20x faster than Meta's current clusters, 9x faster at running the NVIDIA Collective Communication Library (NCCL), and 3x faster at training large-scale NLP workflows.", 'With RSC, models with tens of billions of parameters can finish training in three weeks compared to nine weeks previously.', 'RSC was designed with security and privacy controls to allow M